# Chapter 26: Tasks and Concurrency

This notebook covers structured concurrency in `asyncio`: creating tasks, gathering results, waiting for completion, limiting concurrency with semaphores, and handling timeouts.

## Key Concepts
- **`asyncio.create_task()`**: Schedule a coroutine to run concurrently as a Task
- **`asyncio.gather()`**: Run multiple awaitables concurrently, collect results in order
- **`asyncio.wait()`**: Wait for multiple tasks with flexible completion conditions
- **`asyncio.Semaphore`**: Limit the number of concurrent operations
- **`asyncio.wait_for()` / `asyncio.timeout()`**: Cancel operations that take too long

## Section 1: `asyncio.create_task()`

`create_task()` wraps a coroutine in a `Task` object and schedules it for execution on the event loop. The task starts running as soon as the event loop gets a chance (at the next `await` point).

In [ ]:
import asyncio
import time


async def compute(x: int) -> int:
    """Simulate an async computation."""
    await asyncio.sleep(0)
    return x * 2


async def main() -> None:
    # create_task schedules the coroutine for execution
    task = asyncio.create_task(compute(5))
    print(f"Task type: {type(task).__name__}")
    print(f"Task done before await: {task.done()}")

    # Await the task to get its result
    result = await task
    print(f"Task done after await: {task.done()}")
    print(f"Result: {result}")


asyncio.run(main())

In [ ]:
# Tasks run concurrently — they don't block each other
async def worker(name: str, delay: float) -> str:
    print(f"  {name}: starting (will take {delay}s)")
    await asyncio.sleep(delay)
    print(f"  {name}: done")
    return f"{name} result"


async def main() -> None:
    start = time.perf_counter()

    # Create tasks — they start running immediately
    task1 = asyncio.create_task(worker("Task-1", 0.10))
    task2 = asyncio.create_task(worker("Task-2", 0.05))
    task3 = asyncio.create_task(worker("Task-3", 0.08))

    # Await each task to get its result
    r1 = await task1
    r2 = await task2
    r3 = await task3

    elapsed = time.perf_counter() - start
    print(f"\nResults: {[r1, r2, r3]}")
    print(f"Elapsed: {elapsed:.3f}s (concurrent, not 0.23s)")


asyncio.run(main())

## Section 2: `asyncio.gather()`

`gather()` runs multiple awaitables concurrently and returns their results as a list. The results are always in the **same order** as the arguments, regardless of which coroutine finishes first.

In [ ]:
# gather runs coroutines concurrently and collects results
results: list[int] = []


async def append_value(val: int) -> int:
    await asyncio.sleep(0)
    results.append(val)
    return val


async def main() -> list[int]:
    return await asyncio.gather(
        append_value(1),
        append_value(2),
        append_value(3),
    )


gathered = asyncio.run(main())
print(f"Gathered results: {gathered}")
print(f"Execution order: {sorted(results)}")

In [ ]:
# gather preserves argument order, not completion order
async def delayed(val: int, delay: float) -> int:
    """Return a value after a delay."""
    await asyncio.sleep(delay)
    return val


async def main() -> None:
    # Task 3 finishes first, but appears last in results
    results = await asyncio.gather(
        delayed(1, 0.02),   # Finishes last
        delayed(2, 0.01),   # Finishes second
        delayed(3, 0.00),   # Finishes first
    )
    print(f"Results (argument order): {results}")
    # Always [1, 2, 3] regardless of completion order


asyncio.run(main())

In [ ]:
# gather with dynamic number of coroutines
async def fetch_page(page_num: int) -> dict[str, int | str]:
    """Simulate fetching a page of data."""
    await asyncio.sleep(0.01)
    return {"page": page_num, "status": "ok"}


async def fetch_all_pages(total: int) -> list[dict[str, int | str]]:
    """Fetch multiple pages concurrently."""
    # Use unpacking to pass a generator of coroutines
    return await asyncio.gather(
        *(fetch_page(i) for i in range(1, total + 1))
    )


pages = asyncio.run(fetch_all_pages(5))
for page in pages:
    print(f"  Page {page['page']}: {page['status']}")

## Section 3: Error Handling with `gather()`

By default, if any gathered coroutine raises an exception, `gather()` re-raises it. Use `return_exceptions=True` to capture exceptions as results instead.

In [ ]:
async def maybe_fail(n: int) -> str:
    """Succeed on even numbers, fail on odd."""
    await asyncio.sleep(0)
    if n % 2 != 0:
        raise ValueError(f"odd number: {n}")
    return f"ok-{n}"


async def main() -> None:
    # return_exceptions=True captures errors instead of raising
    results = await asyncio.gather(
        maybe_fail(1),
        maybe_fail(2),
        maybe_fail(3),
        maybe_fail(4),
        return_exceptions=True,
    )
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"  Task {i}: FAILED - {result}")
        else:
            print(f"  Task {i}: OK - {result}")


asyncio.run(main())

## Section 4: `asyncio.wait()`

`wait()` gives you more control than `gather()`. It returns two sets: `done` and `pending`. You can wait for the first task to complete, all tasks, or the first exception.

In [ ]:
# wait() with FIRST_COMPLETED — process results as they arrive
async def timed_task(name: str, delay: float) -> str:
    await asyncio.sleep(delay)
    return f"{name} ({delay}s)"


async def main() -> None:
    tasks: set[asyncio.Task[str]] = {
        asyncio.create_task(timed_task("fast", 0.01)),
        asyncio.create_task(timed_task("medium", 0.05)),
        asyncio.create_task(timed_task("slow", 0.10)),
    }

    print("Processing tasks as they complete:")
    while tasks:
        done, tasks = await asyncio.wait(
            tasks, return_when=asyncio.FIRST_COMPLETED
        )
        for task in done:
            print(f"  Completed: {task.result()}")


asyncio.run(main())

In [ ]:
# wait() with ALL_COMPLETED (the default)
async def main() -> None:
    tasks = [
        asyncio.create_task(timed_task("A", 0.03)),
        asyncio.create_task(timed_task("B", 0.01)),
        asyncio.create_task(timed_task("C", 0.02)),
    ]

    done, pending = await asyncio.wait(tasks, return_when=asyncio.ALL_COMPLETED)
    print(f"Done: {len(done)}, Pending: {len(pending)}")
    for task in done:
        print(f"  Result: {task.result()}")


asyncio.run(main())

## Section 5: Task Cancellation

Tasks can be cancelled by calling `task.cancel()`. The cancelled task will raise `asyncio.CancelledError` at its next `await` point.

In [ ]:
async def long_running() -> str:
    """A task that takes a while."""
    try:
        print("  Long task: working...")
        await asyncio.sleep(10)  # Will be cancelled before this finishes
        return "completed"
    except asyncio.CancelledError:
        print("  Long task: cancelled! Cleaning up...")
        raise  # Re-raise to propagate cancellation


async def main() -> None:
    task = asyncio.create_task(long_running())

    # Let the task start
    await asyncio.sleep(0.01)

    # Cancel the task
    task.cancel()

    try:
        await task
    except asyncio.CancelledError:
        print(f"  Main: task was cancelled")

    print(f"  Task cancelled: {task.cancelled()}")


asyncio.run(main())

## Section 6: Timeouts with `asyncio.wait_for()`

`wait_for()` wraps a coroutine with a timeout. If the coroutine does not complete within the timeout, it is cancelled and `asyncio.TimeoutError` is raised.

In [ ]:
async def slow_operation() -> str:
    """An operation that takes too long."""
    await asyncio.sleep(5)
    return "finally done"


async def main() -> None:
    # Set a timeout of 0.1 seconds
    try:
        result = await asyncio.wait_for(slow_operation(), timeout=0.1)
        print(f"Result: {result}")
    except asyncio.TimeoutError:
        print("Operation timed out after 0.1s")

    # A fast operation completes within the timeout
    async def fast_operation() -> str:
        await asyncio.sleep(0.01)
        return "quick result"

    result = await asyncio.wait_for(fast_operation(), timeout=1.0)
    print(f"Fast result: {result}")


asyncio.run(main())

In [ ]:
# Timeout with retry pattern
async def unreliable_fetch(attempt: int) -> str:
    """Simulates a fetch that sometimes times out."""
    delay = 0.5 if attempt < 3 else 0.01  # Fast on third attempt
    await asyncio.sleep(delay)
    return f"data (attempt {attempt})"


async def fetch_with_retries(
    max_retries: int = 3,
    timeout: float = 0.1,
) -> str:
    """Retry a fetch operation with timeout."""
    for attempt in range(1, max_retries + 1):
        try:
            result = await asyncio.wait_for(
                unreliable_fetch(attempt), timeout=timeout
            )
            print(f"  Attempt {attempt}: success")
            return result
        except asyncio.TimeoutError:
            print(f"  Attempt {attempt}: timed out")
    raise RuntimeError(f"Failed after {max_retries} attempts")


result = asyncio.run(fetch_with_retries())
print(f"Final result: {result}")

## Section 7: Semaphores — Limiting Concurrency

`asyncio.Semaphore` limits the number of coroutines that can access a resource concurrently. This is essential for rate limiting, connection pooling, and preventing resource exhaustion.

In [ ]:
# Semaphore limits concurrent access
max_concurrent: int = 0
current: int = 0


async def limited_task(sem: asyncio.Semaphore, task_id: int) -> None:
    """A task that respects a semaphore limit."""
    global max_concurrent, current
    async with sem:
        current += 1
        if current > max_concurrent:
            max_concurrent = current
        print(f"  Task {task_id}: running (concurrent: {current})")
        await asyncio.sleep(0.01)
        current -= 1


async def main() -> None:
    global max_concurrent, current
    max_concurrent = 0
    current = 0

    # Allow at most 2 tasks to run concurrently
    sem = asyncio.Semaphore(2)
    await asyncio.gather(*(limited_task(sem, i) for i in range(5)))
    print(f"\nMax concurrent tasks: {max_concurrent} (limit was 2)")


asyncio.run(main())

In [ ]:
# Practical example: rate-limited API fetcher
async def fetch_url(url: str, sem: asyncio.Semaphore) -> dict[str, str]:
    """Fetch a URL with rate limiting."""
    async with sem:
        print(f"  Fetching {url}...")
        await asyncio.sleep(0.02)  # Simulate network I/O
        return {"url": url, "status": "200"}


async def main() -> None:
    urls = [f"https://api.example.com/item/{i}" for i in range(6)]

    # Limit to 3 concurrent requests
    sem = asyncio.Semaphore(3)

    start = time.perf_counter()
    results = await asyncio.gather(*(fetch_url(url, sem) for url in urls))
    elapsed = time.perf_counter() - start

    print(f"\nFetched {len(results)} URLs in {elapsed:.3f}s")
    print(f"With concurrency limit of 3")


asyncio.run(main())

## Section 8: Task Groups (Python 3.11+)

`asyncio.TaskGroup` provides structured concurrency — if any task in the group fails, all other tasks are cancelled. This is safer than `gather()` for error-prone operations.

In [ ]:
# TaskGroup for structured concurrency
async def process_item(item: int) -> int:
    """Process a single item."""
    await asyncio.sleep(0.01)
    return item * 10


async def main() -> None:
    results: list[int] = []

    async with asyncio.TaskGroup() as tg:
        tasks = [
            tg.create_task(process_item(i))
            for i in range(5)
        ]

    # All tasks are guaranteed complete here
    results = [t.result() for t in tasks]
    print(f"Results: {results}")


asyncio.run(main())

In [ ]:
# TaskGroup handles errors by cancelling remaining tasks
async def might_fail(n: int) -> int:
    await asyncio.sleep(0.01)
    if n == 3:
        raise ValueError(f"Cannot process {n}")
    return n * 10


async def main() -> None:
    try:
        async with asyncio.TaskGroup() as tg:
            tasks = [tg.create_task(might_fail(i)) for i in range(5)]
    except* ValueError as eg:
        print(f"Caught {len(eg.exceptions)} error(s):")
        for exc in eg.exceptions:
            print(f"  - {exc}")

    # Check which tasks completed vs were cancelled
    for i, t in enumerate(tasks):
        if t.cancelled():
            print(f"  Task {i}: cancelled")
        elif t.exception():
            print(f"  Task {i}: failed")
        else:
            print(f"  Task {i}: result = {t.result()}")


asyncio.run(main())

## Summary

### `asyncio.create_task()`
- Wraps a coroutine in a `Task` and schedules it on the event loop
- Tasks start running at the next `await` point
- Returns a `Task` object you can await, cancel, or inspect

### `asyncio.gather()`
- Runs multiple awaitables concurrently
- Returns results in **argument order**, not completion order
- Use `return_exceptions=True` to capture errors instead of raising

### `asyncio.wait()`
- Returns `(done, pending)` sets of tasks
- Supports `FIRST_COMPLETED`, `ALL_COMPLETED`, `FIRST_EXCEPTION`
- Good for processing results as they arrive

### Timeouts
- `asyncio.wait_for(coro, timeout)` cancels after timeout seconds
- Raises `asyncio.TimeoutError` on expiration
- Combine with retry logic for resilient operations

### Semaphores
- `asyncio.Semaphore(n)` limits concurrency to `n` coroutines
- Use `async with sem:` to acquire and release automatically
- Essential for rate limiting and resource management

### Task Groups (3.11+)
- `asyncio.TaskGroup` provides structured concurrency
- If any task fails, all others are cancelled
- Use `except*` to handle `ExceptionGroup` errors